# Clasificación de Imágenes MRI con RNN

Este notebook explora el uso de redes neuronales recurrentes (RNN) para la clasificación de imágenes MRI de tumores cerebrales. Incluye:

1. Preprocesamiento y conversión de imágenes a secuencias
2. Definición y entrenamiento de un modelo RNN
3. Evaluación de resultados

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Flatten

# Configuración de rutas
DATASET_DIR = "../total/archive/Training"

In [ ]:
# Ejemplo de modelo RNN para imágenes (requiere convertir imágenes a secuencias)
# model = Sequential([
#     SimpleRNN(128, input_shape=(secuencia_largo, secuencia_dim)),
#     Dense(num_classes, activation='softmax')
# ])
# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

Las RNN no son ideales para imágenes, pero este notebook es educativo para explorar su uso en visión computacional.